In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1)
plt.style.use('ggplot')

import pandas as pd
import numpy as np
import random as rnd

from sklearn.cross_validation import KFold, cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../trainval.csv')
validate_=pd.read_csv('../validate.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, #validate.shape, #test.shape

/Users/arc/Codes/DM-Lab/dmlab/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


((485842, 33),)

In [1]:
train = train_.query(DSMCRIT>=14) # 14 = anxiety disorder
validate = validate.query(DSMCRIT>=14) # 14 = anxiety disorder

NameError: name 'train' is not defined

In [8]:
print train['DSMCRIT'].value_counts() 


4     496231
5     412099
7     219372
8     218609
3     203724
9     165378
10    156135
6     128801
2      68405
13     31498
20     28818
11     26109
1      21551
12     19453
0      17613
15     13459
19      7621
17      6774
14      4698
16      3570
18      2222
Name: DSMCRIT, dtype: int64


In [58]:
print validate['SUB2'].value_counts()
print 214199/415452.

1     214199
4      70651
2      56321
3      34764
7      22031
10     10668
5       6818
Name: SUB2, dtype: int64
0.515580620625


In [45]:
#train.describe()
train = train.sample(10000)
validate = validate.sample(6000)
train.shape, validate.shape, validate.head(2)

((10000, 48),
 (6000, 48),
         YEAR  AGE  GENDER  RACE  ETHNIC  MARSTAT  EDUC  EMPLOY  DETNLF  PREG  \
 490431  2013   12       2     5       5        4     3       4       3     2   
 205645  2011    4       2     5       5        1     3       4       2     2   
 
          ...     BENZFLG  TRNQFLG  BARBFLG  SEDHPFLG  INHFLG  OTCFLG  \
 490431   ...           0        0        0         0       0       0   
 205645   ...           0        0        0         0       0       0   
 
         OTHERFLG  ALCDRUG  DSMCRIT  PSYPROB  
 490431         0        3        2        1  
 205645         0        3        8        1  
 
 [2 rows x 48 columns])

In [43]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

drop_list = ['DSMCRIT', 'YEAR', 'STFIPS', 
             'ALCFLG', 'COKEFLG', 'MARFLG', 'HERFLG', 'METHFLG', 'OPSYNFLG', 'PCPFLG', 'HALLFLG', 'MTHAMFLG', 
             'AMPHFLG', 'STIMFLG', 'BENZFLG', 'TRNQFLG', 'BARBFLG', 'SEDHPFLG', 'INHFLG', 'OTCFLG', 'OTHERFLG', 
             'ALCDRUG', #'NUMSUBS'
             ]

X_train = train.drop(drop_list, axis=1)
Y_train = train["DSMCRIT"]
X_validate = validate.drop(drop_list, axis=1)
Y_validate = validate["DSMCRIT"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((10000, 26), (6000, 26))

In [37]:
print X_train.columns.tolist()

['AGE', 'GENDER', 'RACE', 'ETHNIC', 'MARSTAT', 'EDUC', 'EMPLOY', 'DETNLF', 'PREG', 'VET', 'LIVARAG', 'ARRESTS', 'SERVSETA', 'METHUSE', 'PSOURCE', 'DETCRIM', 'NOPRIOR', 'SUB1', 'ROUTE1', 'FREQ1', 'FRSTUSE1', 'SUB2', 'SUB3', 'NUMSUBS', 'IDU', 'PSYPROB']


In [38]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((10000, 174), (6000, 174))

In [39]:
# Logistic Regression
logreg = LogisticRegression(n_jobs=-1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)

l_acc_log = cross_val_score(logreg, X_train, Y_train, cv=5)
acc_log = round(np.mean(l_acc_log), 3)

l_acc_log = ['%.3f' % elem for elem in l_acc_log]
print l_acc_log
print acc_log

['0.681', '0.671', '0.676', '0.671', '0.684']
0.677


In [40]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=200, max_depth=20, n_jobs=-1)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)

l_acc_random_forest = cross_val_score(random_forest, X_train, Y_train, cv=5)
acc_random_forest = round(np.mean(l_acc_random_forest), 3)

l_acc_random_forest = ['%.3f' % elem for elem in l_acc_random_forest]
print l_acc_random_forest
print acc_random_forest

['0.684', '0.673', '0.682', '0.677', '0.688']
0.681


In [46]:
# Linear SVC
linear_svc = LinearSVC(C=0.001)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)

l_acc_linear_svc = cross_val_score(linear_svc, X_train, Y_train, cv=5)
acc_linear_svc = round(np.mean(l_acc_linear_svc), 3)
l_acc_linear_svc = ['%.3f' % elem for elem in l_acc_linear_svc]
print l_acc_linear_svc
print acc_linear_svc

['0.568', '0.547', '0.536', '0.535', '0.564']
0.55


In [42]:
print 'predict-sub2-woflags-newsplit-sample20000'
models = pd.DataFrame({
    'Model': ['Logistic Regression', 
              'Random Forest','Linear SVC'],
    'Cross Validation': [l_acc_log, 
              l_acc_random_forest, l_acc_linear_svc],
    'Cross Validation Mean': [acc_log, 
              acc_random_forest, acc_linear_svc]
    })
models.sort_values(by='Cross Validation Mean', ascending=False)


predict-sub2-woflags-newsplit-sample20000


,Cross Validation,Cross Validation Mean,Model
1,"[0.684, 0.673, 0.682, 0.677, 0.688]",0.681,Random Forest
0,"[0.681, 0.671, 0.676, 0.671, 0.684]",0.677,Logistic Regression
2,"[0.671, 0.662, 0.669, 0.668, 0.671]",0.668,Linear SVC
